In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [11]:
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/dataset_adjusted.csv?token=GHSAT0AAAAAACC4ZCNLSI7IUHBG7JHPMLT2ZGMCCBQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'dataset_adjusted' variable
    dataset_adjusted = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'dataset_adjusted' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/raw/dataset_words.csv?token=GHSAT0AAAAAACC4ZCNKNKB3MZOF4DY7SDEIZGMC73Q"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'words' variable
    words = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'words' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

Dataset downloaded and loaded into 'dataset_adjusted' successfully.
Dataset downloaded and loaded into 'words' successfully.


In [12]:
# Concatenate the vectorized DataFrame with the original dataset
full_dataset = pd.concat([dataset_adjusted, words], axis=1)

In [15]:
if "Date" in full_dataset.columns:
    date_column = full_dataset["Date"]
    print(date_column)
else:
    print("The 'Date' column is not found in the full_dataset.")

           Date        Date
0    1993-05-18  1993-05-18
1    1993-07-07  1993-07-07
2    1993-08-17  1993-08-17
3    1993-09-21  1993-09-21
4    1993-11-16  1993-11-16
..          ...         ...
235  2022-11-02  2022-11-02
236  2022-12-14  2022-12-14
237  2023-02-01  2023-02-01
238  2023-03-22  2023-03-22
239  2023-05-03  2023-05-03

[240 rows x 2 columns]
